In [231]:
import os
import pandas as pd
import datetime
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# 引用 tweetTextHandler.ipynb
import ipynb.fs.defs.TweetTextHandler as tweetTextHandler
import operator
global stop_words
stop_words = stopwords.words('english')

In [265]:
def getWordsDict(texts, category='inRange'):
    global stop_words
    wordDict = dict()
    stopwordDict = dict()
    stopwordTweetCount = 0
    tknzr = TweetTokenizer()

    for txt in texts:
        stopwordFlag = False
        try:
            txt = tweetTextHandler.del_url(txt)
            for token in tknzr.tokenize(txt):
                #token = porter.stem(token.lower())
                if token.isdigit():
                    pass
                elif token in stop_words:
                    stopwordDict[token] = stopwordDict.get(token, 0) + 1
                    stopwordFlag = True
                else:
                    wordDict[token] = wordDict.get(token, 0) + 1
            if stopwordFlag:
                stopwordTweetCount += 1
        except Exception as e:
            #print("Error message:", e)
            pass
    return wordDict, stopwordDict, stopwordTweetCount

In [283]:
class Patient(object):
    totalCount = 0
    diagnosedTimeAndText = dict()
    with open('../0_dataset/diagnosed_tweets.csv') as open_file:
        for line in open_file.readlines():
            line = line.strip().split('\t')
            diagnosedTimeAndText[line[1]] = (line[0], line[2])
       
    def __init__(self, name):
        self.name = name
        self.df = pd.read_csv('../0_dataset/DepressionUsersTweets/'+name,sep='\t',header=None,usecols=[2,3],names=['Date','Text'],quoting=3,error_bad_lines=False,encoding='utf-8',keep_default_na=False)
        self.diagnosedTime = datetime.datetime.strptime(Patient.diagnosedTimeAndText[name][1], "%Y-%m-%d %H:%M:%S")
        self.diagnosedTweet = Patient.diagnosedTimeAndText[name][0]
        
        beforeDiagosedTime = str(self.diagnosedTime - datetime.timedelta(days=90))
        self.inRangeDf = self.df[self.df['Date'].between(beforeDiagosedTime, str(self.diagnosedTime))]
        
        Patient.totalCount += 1 
        
    def displayCount(self):
        print("Total Patients {0}".format(Patient.totalCount))
    
    def displayTweetsCount(self, category='inRange'):
        if category == 'inRange':
            return len(self.inRangeDf)
        elif category == 'all':
            return len(self.df)
        else:
            print('Please indicate category (inRange or all)')
            return False
        
    def getText(self, category='inRange'):
        if category == 'inRange':
            return self.inRangeDf['Text'].values
        elif category == 'all':
            return self.df['Text'].values
        else:
            print('Please indicate category (inRange or all)')
            return False

In [289]:
class Ordinary(object):
    totalCount = 0

    def __init__(self, name):
        self.name = name
        self.df = pd.read_csv('../0_dataset/ordinaryUsersTweets/'+name,sep='\t',header=None,usecols=[2,3],names=['Date','Text'],quoting=3,error_bad_lines=False,encoding='utf-8',keep_default_na=False)
        # The method to get the latest tweet is wrong
        try:
            self.latestTweetTime = datetime.datetime.strptime(str(self.df.iloc[-1]['Date']), "%Y-%m-%d %H:%M:%S")
        except:
            print(name)
        self.latestTweetText = self.df.iloc[-1]['Text']
        
        beforeLatestTweetTime = str(self.latestTweetTime - datetime.timedelta(days=90))
        self.inRangeDf = self.df[self.df['Date'].between(beforeLatestTweetTime, str(self.latestTweetTime))]
        
        Ordinary.totalCount += 1 
        
    def displayCount(self):
        print("Total Ordinarys {0}".format(Ordinary.totalCount))
    
    def displayTweetsCount(self, category='inRange'):
        if category == 'inRange':
            return len(self.inRangeDf)
        elif category == 'all':
            return len(self.df)
        else:
            print('Please indicate category (inRange or all)')
            return False
        
    def getText(self, category='inRange'):
        if category == 'inRange':
            return self.inRangeDf['Text'].values
        elif category == 'all':
            return self.df['Text'].values
        else:
            print('Please indicate category (inRange or all)')
            return False

In [285]:
patients = dict()
ordinarys = dict()

In [286]:
with open('../0_dataset/patient_ids') as r:
    for patient in r.readlines()[:100]:
        patient = patient.strip()
        patients[patient] = Patient(patient)

In [290]:
with open('../0_dataset/ordinary_ids') as r:
    for ordinary in r.readlines()[:100]:
        ordinary = ordinary.strip()
        ordinarys[patient] = Ordinary(ordinary)

1031748547


AttributeError: 'Ordinary' object has no attribute 'latestTweetTime'

In [294]:
test = pd.read_csv('../0_dataset/ordinaryUsersTweets/1031748547',sep='\t',header=None,usecols=[2,3],names=['Date','Text'],quoting=3,error_bad_lines=False,encoding='utf-8')

In [296]:
test.dropna(axis=0, how='any')

,Date,Text
0,2013-04-03 22:27:25,@OfficialTWWorld Amazing :-)
1,2013-04-03 08:50:45,Love this :-) http://t.co/0ODYIyBIqA
2,2013-04-09 09:07:25,#ProudToBeAFanOf both The Wanted and One Direc...
3,2013-04-09 10:48:35,@ohgirlbook story of my life :-)
4,2013-04-12 15:31:24,I'm on a college trip with my school and GOD s...
5,2013-04-12 21:08:41,Go follow @YayaGonzalez115 everyone she follow...
6,2013-04-14 18:09:28,@harrysyogurt All of them...
7,2013-04-14 20:27:04,@KirstyTWx Spot on except for the age... I'm 1...
8,2013-04-14 23:55:02,#iHateWhenPeopleExpectMeTo just give them my w...
9,2013-04-13 12:37:12,@MariaTWFanmily and his lips OMG
